In [1]:
import os
import utilities
from maths_objects import (
    Formulas, DirectoryHierarchies,MarkdownFile, DataSource, FrontMatter, 
    FormulaTable)
import pandas as pd


data_sources = DataSource()


In [2]:
formulas = Formulas(data_sources.formulas_by_year())
df = formulas._unique_state_subject_categories()

# def  my_generator():
#     for item in df.itertuples():
#         formula_table = FormulaTable(formulas)
#         formula_table.state = item.State
#         formula_table.category = item.Category
#         formula_table.subject = item.Subject
#         yield(formula_table)


# for item in my_generator():
#     print (item.category)

for table in formulas.formula_tables():
    display(table._formula_data)

,Formula
0,$a^m \times a^n = a^{m+n}$
1,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$
2,$(a^m)^n = a^{m \times n}$
3,$(ab)^n = a^nb^n$
4,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$
5,$a^0=1$
6,$a^{-n}=\dfrac{1}{a^n}$
7,$\left(\dfrac{a}{b}\right)^{-n} = \left(\dfrac...
8,$a^{\frac{m}{n}} =\sqrt[n]{a^m}$


,Formula
9,$\log_a{(xy)} = \log_a{x} + log_a{y}$
10,$\log_a{\left(\dfrac{x}{y}\right)} = \log_a{x}...
11,$\log_a{x^n} = n\log_a{x}$
12,$\log_a{\left(\dfrac{1}{x}\right)} = -\log_a{x}$
13,$\log_a{x} = \dfrac{\log_b{x}}{\log_b{a}}$


,Formula
14,"$ y=f(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\..."
15,"$ y=kx \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\..."
16,"$ y=k \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
17,"$ y=x^n \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
18,"$ y=kx^n \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\..."
19,"$ y=kf(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
20,"$ y=f(x) + g(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,..."


,Formula
21,"$y=f(x)^n \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
23,"$y=uv \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
25,"$y=\dfrac{u}{v} \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
26,"$y=g(u) \text { where } u = f(x) \,\,\,\,\,\,\..."
28,"$y=sinf(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\..."
30,"$y=cosf(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\..."
32,"$y=tan f(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
34,"$y=e^{f(x)} \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
36,"$y=\ln f(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
38,"$y=a^{f(x)} \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."


,Formula
22,$ {\Large\int} f'(x)[f(x)]^n dx = \dfrac{1}{n+...
24,$ {\Large\int} u \dfrac{dv}{dx} dx=uv-{\Large\...
27,$ {\Large\int_{a}^{b}} f(x) dx \approx \dfrac{...
29,$ {\Large\int} f'(x)cosf(x)dx = sin f(x) + c$
31,$ {\Large\int} f'(x)sin f(x)dx = -cos f(x) + c$
33,$ {\Large\int} f'(x)sec^2 f(x)dx = tan f(x) + c$
35,$ {\Large\int} f'(x)e^{f(x)}dx = e^{f(x)} + c$
37,$ {\Large\int} \dfrac{f'(x)}{f(x)}dx = \ln|f(x...
39,$ {\Large\int} f'(x)a^{f(x)}dx = \dfrac{a^f(x)...
41,$ {\Large\int} \dfrac{f'(x)}{(\ln a) f(x)} = \...


,Formula
42,"$y=sin^{-1}f(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
44,"$ y = sin^{-1} \dfrac{f(x)}{a} \,\,\,\,\,\,\,..."
46,"$y=cos^{-1}f(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
48,"$y=cos^{-1} \dfrac{f(x)}{a} \,\,\,\,\,\,\,\,\,..."
50,"$y=tan^{-1}f(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,..."
52,"$y=tan^{-1}\dfrac{f(x)}{a} \,\,\,\,\,\,\,\,\,\..."
54,"$ \text{Function and its inverse} \,\,\,\,\,\..."


,Formula
43,$ {\Large\int} \dfrac{f'(x)} {\sqrt {1 - (f(x)...
45,$ {\Large\int} \dfrac{f'(x)} {\sqrt {a^2 - (f(...
47,$ {\Large\int} - \dfrac{f'(x)} {\sqrt {1 - (f(...
49,$ {\Large\int} - \dfrac{f'(x)} {\sqrt {a^2 - (...
51,$ {\Large\int} \dfrac{f'(x)} {1 + (f(x))^2} dx...
53,$ {\Large\int} \dfrac{f'(x)} {a^2 + (f(x))^2} ...


,Formula
55,$ \text{Mean} = \dfrac{\text{Sum of scores}}{\...
56,$ \text{The median of n scores is the } \dfrac...
57,$ \text{Range = highest score - lowest score} $
58,$ \text{Interquartile range = Q3 - Q1} $
59,$\text{An outlier is a score with} $\n$ \text ...
60,$ \text{Variance } = \sigma^2 = \dfrac{\sum(x...
61,$ \text{Standard deviation } = \sqrt{\text{var...


,Formula
62,$ \sqrt{xy} = \sqrt{x} \times \sqrt{y} $
63,$ \sqrt{\dfrac{x}{y}} = \dfrac{\sqrt{x}}{\sqrt...


,Formula
64,$A = P(1+r)^n$
65,$ T_n = T_{n-1} + d $
66,$T_n = a + (n-1)d$
67,$S_n = \dfrac{n}{2}[2a + (n-1)d] = \dfrac{n}{2...
68,$T_n = rT_{n-1}$
69,$T_n = ar^{n-1}$
70,$S_n = \dfrac{a(1-r^n)}{1-r} = \dfrac{a(r^n-1)...
71,"$S=\dfrac{a}{1-r},\ \ |r|<1$"


In [6]:
formulas.filter_by_state_subject_category('NSW', 'Year 10', 'Indices').to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment


In [4]:
formulas.filter_by_state_subject_category('NSW', 'Year 9', 'Indices')()

AttributeError: 'DataFrame' object has no attribute 'filter_by_function'

In [13]:
formulas.filter_by_function(
    lambda row : 
    (row['State'] == 'NSW') and 
    (row['Subject'] == 'Year 9') and 
    (row['Category'] == 'Indices')
    ).to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment
0,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Multiplying terms with same base,NaN,$a^m \times a^n = a^{m+n}$,True,False,NaN
1,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Dividing terms with same base,NaN,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,True,False,NaN
2,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Power of a power,NaN,$(a^m)^n = a^{m \times n}$,True,False,NaN
3,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of products,NaN,$(ab)^n = a^nb^n$,True,False,NaN
4,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,True,False,NaN
5,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Power of zero,NaN,$a^0=1$,True,False,NaN
6,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Negative powers,NaN,$a^{-n}=\dfrac{1}{a^n}$,True,False,NaN
7,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Negative powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^{-n} = \left(\dfrac...,True,False,NaN
8,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Fractional powers,NaN,$a^{\frac{m}{n}} =\sqrt[n]{a^m}$,True,False,NaN


In [22]:

data_sources = DataSource()

formulas_by_year = data_sources.formulas_by_year()
state_subject_sort_orders = data_sources._state_subject_sort_orders()


# Add the subject Sort order (representing the sort order for the
# subject by given state) to the formulas data
return_data = state_subject_sort_orders.merge(
    right=formulas_by_year,
    how='inner',
    left_on=['Sort state', 'Sort subject'],
    right_on=['State', 'Subject']
)
return_data = return_data.rename(
    columns={
        'State subject sort order': 'Data sort index'})
return_data = return_data.drop(columns=['Sort state',
                                        'Sort subject'])

return_data = return_data.merge(
    right=state_subject_sort_orders, 
    how='inner', left_on=['State'], right_on=['Sort state']
)
return_data = return_data[return_data['State subject sort order'] >=
                        return_data['Data sort index']]
return_data = return_data.drop(columns=['Data sort index',
                                    'State subject sort order',
                                    'Sort state', 'Subject'])
return_data = return_data.rename(columns={
    'Sort subject': 'Subject'})


In [23]:
temp_file = data_sources.website_creator_directory() + os.path.sep + 'Temp.csv'

return_data.to_csv(temp_file)


In [2]:
data_sources = DataSource()
data_sources._state_subject_sort_orders()

,State,Subject,State subject sort order
0,NSW,Year 9,0
1,NSW,Year 11 Adv,1
2,NSW,Year 12 Adv,2
3,NSW,Year 12 Ext 1,3


In [2]:
a = my_dirs.truncate_to_path_length(4).unique().reset_sort_index()

display(a.to_dataframe())

b = a.filter_by_path_start(['NSW', 'Topics'])
display(b.to_dataframe())
display(a.to_dataframe())

,Level_0,Level_1,Level_2,Level_3
0,NSW,Topics,By year,Year 9
1,NSW,Topics,By year,Year 11 Adv
2,NSW,Topics,By year,Year 12 Adv
3,NSW,Topics,By year cumulative,Year 9
4,NSW,Topics,By year cumulative,Year 11 Adv
5,NSW,Topics,By year cumulative,Year 12 Adv
6,NSW,Formulas,By year,Year 9
7,NSW,Formulas,By year,Year 12 Adv
8,NSW,Formulas,By year,Year 12 Ext 1
9,NSW,Formulas,By year cumulative,Year 12 Adv


,Level_0,Level_1,Level_2,Level_3
0,NSW,Topics,By year,Year 9
1,NSW,Topics,By year,Year 11 Adv
2,NSW,Topics,By year,Year 12 Adv
3,NSW,Topics,By year cumulative,Year 9
4,NSW,Topics,By year cumulative,Year 11 Adv
5,NSW,Topics,By year cumulative,Year 12 Adv


,Level_0,Level_1,Level_2,Level_3
0,NSW,Topics,By year,Year 9
1,NSW,Topics,By year,Year 11 Adv
2,NSW,Topics,By year,Year 12 Adv
3,NSW,Topics,By year cumulative,Year 9
4,NSW,Topics,By year cumulative,Year 11 Adv
5,NSW,Topics,By year cumulative,Year 12 Adv
6,NSW,Formulas,By year,Year 9
7,NSW,Formulas,By year,Year 12 Adv
8,NSW,Formulas,By year,Year 12 Ext 1
9,NSW,Formulas,By year cumulative,Year 12 Adv


In [34]:

# Incorporate below in index file creation to make it clearer how levels are 
# iterated.  Can I improve further by creating an iterator.  In the 
# Web page hierarchy class?

dir_levels = school_formulas.by_year_dirs()
current_dir_levels = dir_levels
for column in reversed(dir_levels.columns):
    display(current_dir_levels)
    current_dir_levels = current_dir_levels.drop(columns=column)

,State,Formula subcategory 1,Formula subcategory 2,Subject
0,NSW,Formulas,By Year,Year 9
9,NSW,Formulas,By Year,Year 11 Adv
21,NSW,Formulas,By Year,Year 12 Adv
42,NSW,Formulas,By Year,Year 12 Ext 1


,State,Formula subcategory 1,Formula subcategory 2
0,NSW,Formulas,By Year
9,NSW,Formulas,By Year
21,NSW,Formulas,By Year
42,NSW,Formulas,By Year


,State,Formula subcategory 1
0,NSW,Formulas
9,NSW,Formulas
21,NSW,Formulas
42,NSW,Formulas


,State
0,NSW
9,NSW
21,NSW
42,NSW


In [7]:
def create_index_files(base_dir, dir_levels,site_hierarchies,  
                       front_matter=None):
    """Creates _index.md files recursively at each level of the directory
    structures formed by joining base_dir string and each element of 
    dir_levels.  The content of each file is set based  on the content
    of the front_matter dictionary and optionally includes a weight
    (sort order) as stored in site_hierarhcies.

    Args:
        base_dir (string): The base direcory that together with dir_levels
            represent the directories where the _index files
            are created
        dir_levels (dataframe): The various directory levels that are joined
            to base_dir
        site_hierarchies (WebPageHierarhies): Object storing the web page 
            hierarchies and sort order
                        
        front_matter (dictionary, optional): Optional front matter text to be
            stored in the _index.md files.
            Defaults to None.
    """ 
    if front_matter is None:
        front_matter = {}
    for i in range(1, len(dir_levels.columns)+1):
        # Looping through various levels in directory structure
        subdirs_df = dir_levels.iloc[:, :i].drop_duplicates()
        for _, row in subdirs_df.iterrows():
            file_name = (base_dir + os.path.sep +
                         os.path.sep.join(row) + os.path.sep + '_index.md')
            if not os.path.isfile(file_name):
                # Don't overwrite file if it exists
                # Use a copy on each loop to start fresh and not
                # carry over any details from previos loop
                front_matter_to_write = front_matter.copy()
                if sort_orders_df is not None:
                    sort_order = lookup_list_in_df(
                        sort_orders_df, list(row.values))
                    if sort_order is not None:
                        front_matter_to_write['weight'] = sort_order + 1
                string_to_write = get_front_matter_string(
                    front_matter_to_write)
                with open(file_name, "w", encoding="utf-8") as text_file:
                    text_file.write(string_to_write)

1